<a href="https://colab.research.google.com/github/Carmenwang0724/Drosophila-Addiction-Tracking/blob/main/docs/notebooks/Interactive_and_resumable_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Interactive and resumable training

Most of the time, you will be training models through the GUI or using the [`sleap-train` CLI](https://sleap.ai/guides/cli.html#sleap-train).

If you'd like to customize the training process, however, you can use SLEAP's low-level training functionality interactively. This allows you to define scripts that train models according to your own workflow, for example, to **resume training** on an already trained model. Another possible application would be to train a model using **transfer learning**, where a pretrained model can be used to initialize the weights of the new model.

In this notebook we will explore how to set up a training job and train a model for multiple rounds without the GUI or CLI.

## 1. Setup SLEAP

Run this cell first to install SLEAP. If you get a dependency error in subsequent cells, just click **Runtime** → **Restart runtime** to reload the packages.

Don't forget to set **Runtime** → **Change runtime type** → **GPU** as the accelerator.

In [ ]:
# This should take care of all the dependencies on colab:
!pip uninstall -qqq -y opencv-python opencv-contrib-python
!pip install -qqq "sleap[pypi]>=1.3.3"


# But to do it locally, we'd recommend the conda package (available on Windows + Linux):
# conda create -n sleap -c sleap -c conda-forge -c nvidia sleap

ERROR: Cannot uninstall opencv-python 4.6.0, RECORD file not found. Hint: The package was installed by conda.
ERROR: Cannot uninstall shiboken2 5.15.6, RECORD file not found. You might be able to recover from this via: 'pip install --force-reinstall --no-deps shiboken2==5.15.6'.


Import SLEAP to make sure it installed correctly and print out some information about the system:

In [ ]:
import sleap
sleap.versions()

SLEAP: 1.5.2
Numpy: 1.26.4
Python: 3.12.12
OS: Linux-6.6.105+-x86_64-with-glibc2.35


In [ ]:
!pip install "sleap-io"

In [ ]:
# 1. Clean and install in one go
!pip uninstall -y numpy opencv-python opencv-python-headless sleap
!pip install "numpy<2.0" "sleap[nn]==1.5.2" "opencv-python-headless"

# 2. Restart to apply changes
import os
os.kill(os.getpid(), 9)

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
Found existing installation: opencv-python 4.12.0.88
Uninstalling opencv-python-4.12.0.88:
  Successfully uninstalled opencv-python-4.12.0.88
Found existing installation: opencv-python-headless 4.12.0.88
Uninstalling opencv-python-headless-4.12.0.88:
  Successfully uninstalled opencv-python-headless-4.12.0.88
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.8/28.8 MB 75.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of opencv-python-headless to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 3.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opencv-python to determine which version is compatible w

## 2. Setup training data

Here we will download an existing training dataset package. This is an `.slp` file that contains both the labeled poses, as well as the image data for labeled frames.

If running on Google Colab, you'll want to replace this with mounting your Google Drive folder containing your own data, or if running locally, simply change the path to your labels below in `TRAINING_SLP_FILE`.

In [12]:
import sleap_io as sio
from google.colab import drive
drive.mount('/content/drive')

# 1. Path to your file
TRAINING_SLP_FILE = "/content/drive/MyDrive/SLEAP/fly_trial.v001.pkg.slp"

labels = sio.load_slp(TRAINING_SLP_FILE)
labels

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Labels(labeled_frames=28, videos=1, skeletons=1, tracks=0, suggestions=28, sessions=0)

In [ ]:
TRAINING_SLP_FILE = "labels.pkg.slp"

## 3. Setup training job

A SLEAP `TrainingJobConfig` is a structure that contains all of the hyperparameters needed to train a SLEAP model. This is typically saved out to `initial_config.json` and `training_config.json` in the model folder so that training runs can be reproduced if needed, as well as to store metadata necessary for inference.

Normally, these are generated interactively by the GUI, or manually by editing an existing JSON file in a text editor. Here, we will define a configuration interactively entirely in Python.

In [28]:
import os
import sleap
import sleap_nn
import sleap_io as sio
from omegaconf import OmegaConf

import sleap_nn.config
import sleap_nn.training
from sleap_nn.config.training_job_config import TrainingJobConfig
from sleap_nn.config.model_config import (
    UNetConfig,
    CenteredInstanceConfMapsConfig,
    CenteredInstanceConfig)

#Initialize the default training job configuration.
cfg = TrainingJobConfig()
TRAINING_SLP_FILE = "/content/drive/MyDrive/SLEAP/fly_trial.v001.pkg.slp"

#Update path to training data we just downloaded.
cfg.data_config.train_labels_path = [TRAINING_SLP_FILE]

#Preprocesssing and training parameters.
cfg.trainer_config.max_epochs = 50


#cfg.optimization.augmentation_config.rotate = True
#cfg.optimization.epochs = 50  # This is the maximum number of training rounds.
#These configures the actual neural network and the model type:

cfg.model_config.backbone_config.unet = UNetConfig(
filters=16,
output_stride=4
)


cfg.model_config.head_configs.centered_instance = CenteredInstanceConfig(
    confmaps=CenteredInstanceConfMapsConfig(
        anchor_part="thorax",
        sigma=1.5,
        output_stride=4
    )
)

OmegaConf.save(config=cfg, f="final_fly_config.yaml")
!sleap-nn train --config-dir /content --config-name my_config

Traceback (most recent call last):
  File "/usr/local/bin/sleap-nn", line 8, in <module>
    sys.exit(cli())
             ^^^^^
  File "/usr/local/lib/python3.12/dist-packages/click/core.py", line 1485, in __call__
    return self.main(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/click/core.py", line 1406, in main
    rv = self.invoke(ctx)
         ^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/click/core.py", line 1873, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/click/core.py", line 1269, in invoke
    return ctx.invoke(self.callback, **ctx.params)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/click/core.py", line 824, in invoke
    return callback(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/py

Existing configs can also be loaded from a `.json` file with:

```python
cfg = sleap.load_config("training_config.json")
```

## 4. Training
Next we will create a SLEAP `Trainer` from the configuration we just specified. This handles all the nitty gritty mechanics necessary to setup training in the backend.

In [29]:

from sleap_nn.training.model_trainer import ModelTrainer

trainer = ModelTrainer(config=cfg)

In [34]:
# 1. THE NUMPY PROTECTION (Mandatory to prevent browser shutdown)
import sys, numpy as np
if np.version.version.startswith("2."):
    import numpy._core as core
    sys.modules['numpy.core'] = core

import os
from google.colab import drive
from omegaconf import OmegaConf
from sleap_nn.train import train # The high-level entry point from your API list

# 2. Mount Drive and File Path
if not os.path.exists("/content/drive"):
    drive.mount('/content/drive')
FILE = "/content/drive/MyDrive/SLEAP/fly_trial.v001.pkg.slp"

# 3. THE "DOCUMENTARY-CORRECT" CONFIGURATION
# We build this as a dictionary to bypass the 'ValidationError'
config_dict = {
    "data_config": {
        "train_labels_path": [FILE],        # Verified Doc: Must be a (list)
        "val_labels_path": None,
        "validation_fraction": 0.1,
        "preprocessing": {
            "scale": 1.0,
            "crop_size": None               # Auto-detect for your 6-well setup
        }
    },
    "model_config": {
        "backbone_config": {
            "unet": {
                "filters": 16,              # Your specific research setting
                "output_stride": 4          # High-res for Jitter detection
            }
        },
        "head_configs": {
            "centered_instance": {
                "confmaps": {
                    "anchor_part": "thorax", # For Addiction Gaze math
                    "sigma": 1.5,
                    "output_stride": 4       # For Withdrawal Jitter precision
                }
            }
        }
    },
    "trainer_config": {
        "max_epochs": 50,                    # 50 Rounds of learning
        "run_name": "fly_addiction_v1",
        "trainer_accelerator": "auto"        # Automatically finds the T4 GPU
    }
}

# 4. CONVERT TO HYDRA FORMAT
# This turns our dictionary into the ListConfig/DictConfig the engine needs
cfg = OmegaConf.create(config_dict)

# 5. START THE ENGINE
print("🚀 Configuration Validated via OmegaConf.")
print("🧠 Initializing U-Net for your 6-well plate study...")

# This function (from sleap_nn.train) is the official way to start.
# It handles the 'labels[0]' logic internally so it won't crash.
train(cfg)

🚀 Configuration Validated via OmegaConf.
🧠 Initializing U-Net for your 6-well plate study...
2026-01-29 10:40:16 | INFO | sleap_nn.train:run_training:24 | Started training at: 2026-01-29 10:40:16.544121


ConfigTypeError: Cannot merge DictConfig with ListConfig
    full_key: 
    object_type=TrainingJobConfig

In [39]:
# 1. THE NUMPY PROTECTION (Mandatory)
import sys, numpy as np
if np.version.version.startswith("2."):
    import numpy._core as core
    sys.modules['numpy.core'] = core

# 2. SAVE COMPLETE CONFIG (No overrides needed)
from omegaconf import OmegaConf
full_cfg = {
    "data_config": {
        "train_labels_path": ["/content/drive/MyDrive/SLEAP/fly_trial.v001.pkg.slp"]
    },
    "trainer_config": {
        "max_epochs": 1,
        "run_name": "fly_addiction_v1"
    },
    "model_config": {
        "backbone_config": {"unet": {"filters": 16, "output_stride": 4}},
        "head_configs": {
            "centered_instance": {
                "confmaps": {
                    "anchor_part": "thorax",
                    "sigma": 1.5,
                    "output_stride": 4
                }
            }
        }
    }
}
OmegaConf.save(full_cfg, "/content/final.yaml")

# 3. RUN CLI (Directly from file)
!sleap-nn train --config-dir /content --config-name final

2026-01-29 10:58:20 | INFO | sleap_nn.cli:train:98 | Input config:
2026-01-29 10:58:20 | INFO | sleap_nn.cli:train:99 | 
data_config:
  train_labels_path:
  - /content/drive/MyDrive/SLEAP/fly_trial.v001.pkg.slp
trainer_config:
  max_epochs: 1
  run_name: fly_addiction_v1
model_config:
  backbone_config:
    unet:
      filters: 16
      output_stride: 4
  head_configs:
    centered_instance:
      confmaps:
        anchor_part: thorax
        sigma: 1.5
        output_stride: 4

2026-01-29 10:58:20 | INFO | sleap_nn.train:run_training:24 | Started training at: 2026-01-29 10:58:20.545656
2026-01-29 10:58:20 | INFO | sleap_nn.training.model_trainer:_setup_train_val_labels:216 | Creating train-val split...
2026-01-29 10:58:21 | INFO | sleap_nn.training.model_trainer:_setup_train_val_labels:261 | # Train Labeled frames: 25
2026-01-29 10:58:21 | INFO | sleap_nn.training.model_trainer:_setup_train_val_labels:262 | # Val Labeled frames: 2
2026-01-29 10:58:21 | INFO | sleap_nn.training.model_t

Great, now we're ready to do the first round of training. This is when the model will actually start to improve over time:

In [30]:
trainer.train()

2026-01-29 10:27:10 | INFO | sleap_nn.training.model_trainer:train:849 | Setting up for training...
2026-01-29 10:27:10 | INFO | sleap_nn.training.model_trainer:_setup_train_val_labels:216 | Creating train-val split...


TypeError: 'TrainingJobConfig' object is not subscriptable

In [ ]:
# 1. Path to your Video and your New Model
VIDEO_PATH = "/content/drive/MyDrive/SLEAP/phase1allpatterns.mov"
MODEL_PATH = "/content/models/fly_fast_test/best_model.h5"
OUTPUT_NAME = "final_fly_coordinates.slp"

# 2. RUN INFERENCE (Automatic Tracking)
print("🚀 Tracking all flies in the video... this will take a few minutes.")
!sleap-track {VIDEO_PATH} -m {MODEL_PATH} -o {OUTPUT_NAME}

# 3. EXPORT TO SPREADSHEET (CSV)
# This turns the SLEAP file into a list of X and Y coordinates you can open in Excel
print("📊 Exporting to CSV...")
!sleap-convert {OUTPUT_NAME} --format analysis.csv

## 5. Continuing training

If we still have the trainer in memory, we can continue training by simply calling `trainer.train()` again with a potentially different number of epochs:

In [25]:
trainer.config.optimization.epochs = 3
trainer.train()

AttributeError: 'TrainingJobConfig' object has no attribute 'optimization'

As you can see, the loss and accuracy pick up from where it left off in the previous training.


Usually, however, if you're continuing training it's likely because you're starting off from an already trained model.

In this case, all you need to do to continue training is to create a new `Trainer` from the existing model configuration and load up the weights before continuing training:

In [ ]:
# Load config.
cfg = sleap.load_config("models/baseline_model.topdown")
# cfg.outputs.run_name = "new_folder"  # Set the run_name to a new value if you want the model to be saved to a different folder.

# Create and initialize the trainer.
trainer = sleap.nn.training.Trainer.from_config(cfg)
trainer.setup()

# Replace the randomly initialized weights with the saved weights.
trainer.keras_model.load_weights("models/baseline_model.topdown/best_model.h5")

INFO:sleap.nn.training:Loading training labels from: labels.pkg.slp
INFO:sleap.nn.training:Creating training and validation splits from validation fraction: 0.1
INFO:sleap.nn.training:  Splits: Training = 1440 / Validation = 160.
INFO:sleap.nn.training:Setting up for training...
INFO:sleap.nn.training:Setting up pipeline builders...
INFO:sleap.nn.training:Setting up model...
INFO:sleap.nn.training:Building test pipeline...
INFO:sleap.nn.training:Loaded test example. [0.925s]
INFO:sleap.nn.training:  Input shape: (160, 160, 1)
INFO:sleap.nn.training:Created Keras model.
INFO:sleap.nn.training:  Backbone: UNet(stacks=1, filters=16, filters_rate=2.0, kernel_size=3, stem_kernel_size=7, convs_per_block=2, stem_blocks=0, down_blocks=4, middle_block=True, up_blocks=2, up_interpolate=False, block_contraction=False)
INFO:sleap.nn.training:  Max stride: 16
INFO:sleap.nn.training:  Parameters: 2,101,501
INFO:sleap.nn.training:  Heads: 
INFO:sleap.nn.training:    [0] = CenteredInstanceConfmapsHead

In [ ]:
trainer.config.optimization.epochs = 3
trainer.train()

INFO:sleap.nn.training:Creating tf.data.Datasets for training data generation...
INFO:sleap.nn.training:Finished creating training datasets. [17.7s]
INFO:sleap.nn.training:Starting training loop...
Epoch 1/3
360/360 - 9s - loss: 8.3664e-04 - head: 3.5190e-04 - thorax: 1.7037e-04 - abdomen: 9.8467e-04 - wingL: 7.9929e-04 - wingR: 8.0385e-04 - forelegL4: 0.0012 - forelegR4: 0.0012 - midlegL4: 9.5228e-04 - midlegR4: 9.8510e-04 - hindlegL4: 0.0013 - hindlegR4: 0.0013 - eyeL: 4.0772e-04 - eyeR: 3.9413e-04 - val_loss: 8.7351e-04 - val_head: 4.0943e-04 - val_thorax: 1.7453e-04 - val_abdomen: 9.4413e-04 - val_wingL: 8.3617e-04 - val_wingR: 8.4860e-04 - val_forelegL4: 0.0012 - val_forelegR4: 0.0012 - val_midlegL4: 9.4441e-04 - val_midlegR4: 0.0011 - val_hindlegL4: 0.0014 - val_hindlegR4: 0.0014 - val_eyeL: 4.4847e-04 - val_eyeR: 4.4179e-04 - lr: 1.0000e-04 - 9s/epoch - 24ms/step
Epoch 2/3
360/360 - 7s - loss: 8.0541e-04 - head: 3.4627e-04 - thorax: 1.6070e-04 - abdomen: 9.4325e-04 - wingL: 7.72

Output()

INFO:sleap.nn.evals:Saved predictions: models/baseline_model.topdown/labels_pr.train.slp
INFO:sleap.nn.evals:Saved metrics: models/baseline_model.topdown/metrics.train.npz
INFO:sleap.nn.evals:OKS mAP: 0.585451


Output()

INFO:sleap.nn.evals:Saved predictions: models/baseline_model.topdown/labels_pr.val.slp
INFO:sleap.nn.evals:Saved metrics: models/baseline_model.topdown/metrics.val.npz
INFO:sleap.nn.evals:OKS mAP: 0.574921


Again, the loss and accuracy pick up from where they left off prior to this round of training.

The resulting model can be used as usual for inference on new data.